<a href="https://colab.research.google.com/github/MKolaksazov/Data-Science/blob/main/Colab%20code%20(graphs)/Tukey_test_letters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import pandas as pd
import numpy as np
import os

# Get the list of files in the current directory
files = os.listdir('.')

# Find the CSV file
csv_file = [file for file in files if file.endswith('.csv')][0]

# прочитане директно от изхода на приложението за обработка на сурови данни от апарата
df = pd.read_csv(csv_file, delimiter='\t', decimal='.', skiprows=0)
csv_file

'Ms_2509_bca_all.csv'

In [61]:
method = 'TPC'

group_order = ['n1', 'n2', 'n3', 'n4', 'n5', 'n6', 'n7', 'n8', 'n9']

In [62]:
df.head()

,groups,method,rep1,rep2
0,n1,GOX,1.790984,1.659836
1,n2,GOX,1.028689,1.020492
2,n3,GOX,1.250000,1.299180
3,n4,GOX,1.381148,1.217213
4,n5,GOX,1.364754,1.282787


In [63]:
def rep_to_numeric(d):
  # Convert 'rep1' and 'rep2' columns to numeric, handling errors
  d['rep1'] = pd.to_numeric(d['rep1'], errors='coerce')
  d['rep2'] = pd.to_numeric(d['rep2'], errors='coerce')

  # Check for missing values in 'rep1' and 'rep2'
  print(d.isnull().sum())
  d = d.dropna();
  print(d.isnull().sum())

  return d

In [64]:
def methods_to_new_cols():
  for method in df_long['method'].unique():
    df_long[method] = df_long[df_long['method'] == method]['value']
    print ('ready')

In [65]:
for col in df.columns:
  if col == 'rep1':
    df = rep_to_numeric(df)
    df_long = pd.melt(df, id_vars=['groups', 'method'], value_vars=['rep1', 'rep2'], var_name='replicate', value_name='value')
    methods_to_new_cols()
    print('true')

groups    0
method    0
rep1      0
rep2      0
dtype: int64
groups    0
method    0
rep1      0
rep2      0
dtype: int64
ready
ready
ready
ready
ready
true


In [66]:
display(df_long.head())

,groups,method,replicate,value,GOX,PM,FRAP,MDA,TPC
0,n1,GOX,rep1,1.790984,1.790984,NaN,NaN,NaN,NaN
1,n2,GOX,rep1,1.028689,1.028689,NaN,NaN,NaN,NaN
2,n3,GOX,rep1,1.250000,1.250000,NaN,NaN,NaN,NaN
3,n4,GOX,rep1,1.381148,1.381148,NaN,NaN,NaN,NaN
4,n5,GOX,rep1,1.364754,1.364754,NaN,NaN,NaN,NaN


In [67]:
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.stats.multicomp as mc

In [68]:
data = pd.DataFrame({
    'Variant': df_long.pop('groups'),
    'Value': df_long.pop(method)
})

data.dropna(inplace=True)

In [69]:
display(data)

,Variant,Value
36,n1,4.243542
37,n2,3.828413
38,n3,4.704797
39,n4,5.119926
40,n5,5.442804
41,n6,3.459410
42,n7,4.797048
43,n8,5.027675
44,n9,5.396679
81,n1,4.243542


In [70]:
# define columns for data and then grouping
col_to_group='Variant'
col_for_data='Value'

#Now take teh data and regroup for anova
samples = [cols[1] for cols in data.groupby(col_to_group)[col_for_data]]

# Предполагаме, че samples е списък от pandas Series
# Пример: samples = [pd.Series([1, 2]), pd.Series([3, 4, 5]), pd.Series([6])]

# Извличаме всички стойности от всички Series в един плосък списък
all_values = []
for series in samples:
    all_values.extend(series.tolist())

# Конвертираме плоския списък в numpy масив
samples = np.array(all_values, dtype='float32')

print(samples.shape, data.shape, data)


#I am not sure how this works but it makes an numpy array for each group
samples = np.asarray(samples).astype('float')
f_val, p_val = stats.f_oneway(*samples)
# I am not sure what this star does but this passes all the numpy arrays correctly
#print(samples)
# this if statement can be uncommmented if you don't won't to go furhter with out p<0.05
#if p_val<0.05:    #If the p value is less than 0.05 it then does the tukey
mod = mc.MultiComparison(data[col_for_data].astype('float'), data[col_to_group])
thsd=mod.tukeyhsd()

#this is a function to do Piepho method.
#AN Alogrithm for a letter based representation of al-pairwise comparisons.
tot=len(thsd.groupsunique)
#print(thsd.groupsunique)
#make an empty dataframe that is a square matrix of size of the groups. #set first column to 1
df_ltr=pd.DataFrame(np.nan, index=np.arange(tot),columns=np.arange(tot))
#df_ltr.iloc[:,0]=1
df_ltr[df_ltr.columns[0]] = 1
#df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)

count=0
#df_nms = pd.DataFrame('', index=np.arange(tot), columns=['names'])

# I make a dummy dataframe to put axis labels into.  sd stands for signifcant difference

for i in np.arange(tot):   #I loop through and make all pairwise comparisons.
    for j in np.arange(i+1,tot):
        if thsd.reject[count]:
            for cn in np.arange(tot):
                #print('i=',i,'j=',j,'cn=', cn, df_ltr)
                if df_ltr.iloc[i,cn]==1 and df_ltr.iloc[j,cn]==1:
                  #If the column contains both i and j shift and duplicat
                    df_ltr=pd.concat([df_ltr.iloc[:,:cn+1],df_ltr.iloc[:,cn+1:].T.shift().T],axis=1)
                    #print('i=',i,'j=',j,'cn=', cn, df_ltr)
                    df_ltr[df_ltr.columns[cn+1]]=df_ltr[df_ltr.columns[cn]]
                    df_ltr.iloc[i,cn]=0
                    df_ltr.iloc[j,cn+1]=0
                    print('i=',i,'j=',j,'cn=', cn, df_ltr)
                #Now we need to check all columns for abosortpion.
                for cleft in np.arange(len(df_ltr.columns)-1):
                    for cright in np.arange(cleft+1,len(df_ltr.columns)):
                        if (df_ltr[df_ltr.columns[cleft]].isna()).all()==False and (df_ltr[df_ltr.columns[cright]].isna()).all()==False:
                            if (df_ltr[df_ltr.columns[cleft]]>=df_ltr[df_ltr.columns[cright]]).all()==True:
                                print('ri=',i,'j=',j,'cn=', cn, "cright", cright, df_ltr)
                                df_ltr[df_ltr.columns[cright]]=0

                                df_ltr=pd.concat([df_ltr[df_ltr.columns[:cright]],df_ltr[df_ltr.columns[cright:]].T.shift(-1).T],axis=1)
                                print('ri=',i,'j=',j,'cn=', cn, "cright", cright, df_ltr)
                            if (df_ltr[df_ltr.columns[cleft]]<=df_ltr[df_ltr.columns[cright]]).all()==True:
                                df_ltr[df_ltr.columns[cleft]]=0
                                print('li=',i,'j=',j,'cn=', cn, "cleft", cleft, df_ltr)
                                df_ltr=pd.concat([df_ltr[df_ltr.columns[:cleft]],df_ltr[df_ltr.columns[cleft:]].T.shift(-1).T],axis=1)
                                print('li=',i,'j=',j,'cn=', cn, "cleft", cleft, df_ltr)

        count+=1
print ("after loop: ", df_ltr)
#I sort so that the first column becomes A
df_ltr=df_ltr.sort_values(by=list(df_ltr.columns),axis=1,ascending=False)
print ("after sorting: ", df_ltr)
# I assign letters to each column
for cn in np.arange(len(df_ltr.columns)):
    df_ltr[df_ltr.columns[cn]]=df_ltr[df_ltr.columns[cn]].replace(1,chr(97+cn))
    df_ltr[df_ltr.columns[cn]]=df_ltr[df_ltr.columns[cn]].replace(0,'')
    df_ltr[df_ltr.columns[cn]]=df_ltr[df_ltr.columns[cn]].replace(np.nan,'')
    #print(df_ltr.columns[cn])

#I put all the letters into one string
df_ltr=df_ltr.astype(str)
df_ltr.sum(axis=1)
df_ltr



(18,) (18, 2)    Variant     Value
36      n1  4.243542
37      n2  3.828413
38      n3  4.704797
39      n4  5.119926
40      n5  5.442804
41      n6  3.459410
42      n7  4.797048
43      n8  5.027675
44      n9  5.396679
81      n1  4.243542
82      n2  3.874539
83      n3  4.381919
84      n4  4.797048
85      n5  5.073801
86      n6  3.413284
87      n7  5.027675
88      n8  4.105166
89      n9  4.012915


/usr/local/lib/python3.12/dist-packages/scipy/stats/_axis_nan_policy.py:573: SmallSampleWarning: all input arrays have length 1.  f_oneway requires that at least one input has length greater than 1.
  if is_too_small(samples, kwds):
/tmp/ipython-input-2717713097.py:24: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  f_val, p_val = stats.f_oneway(*samples)


i= 4 j= 5 cn= 0    0  1   2   3   4   5   6   7   8
0  1  1 NaN NaN NaN NaN NaN NaN NaN
1  1  1 NaN NaN NaN NaN NaN NaN NaN
2  1  1 NaN NaN NaN NaN NaN NaN NaN
3  1  1 NaN NaN NaN NaN NaN NaN NaN
4  0  1 NaN NaN NaN NaN NaN NaN NaN
5  1  0 NaN NaN NaN NaN NaN NaN NaN
6  1  1 NaN NaN NaN NaN NaN NaN NaN
7  1  1 NaN NaN NaN NaN NaN NaN NaN
8  1  1 NaN NaN NaN NaN NaN NaN NaN
after loop:     0  1   2   3   4   5   6   7   8
0  1  1 NaN NaN NaN NaN NaN NaN NaN
1  1  1 NaN NaN NaN NaN NaN NaN NaN
2  1  1 NaN NaN NaN NaN NaN NaN NaN
3  1  1 NaN NaN NaN NaN NaN NaN NaN
4  0  1 NaN NaN NaN NaN NaN NaN NaN
5  1  0 NaN NaN NaN NaN NaN NaN NaN
6  1  1 NaN NaN NaN NaN NaN NaN NaN
7  1  1 NaN NaN NaN NaN NaN NaN NaN
8  1  1 NaN NaN NaN NaN NaN NaN NaN
after sorting:     1  0   2   3   4   5   6   7   8
0  1  1 NaN NaN NaN NaN NaN NaN NaN
1  1  1 NaN NaN NaN NaN NaN NaN NaN
2  1  1 NaN NaN NaN NaN NaN NaN NaN
3  1  1 NaN NaN NaN NaN NaN NaN NaN
4  1  0 NaN NaN NaN NaN NaN NaN NaN
5  0  1 NaN NaN NaN

,1,0,2,3,4,5,6,7,8
0,a,b,,,,,,,
1,a,b,,,,,,,
2,a,b,,,,,,,
3,a,b,,,,,,,
4,a,,,,,,,,
5,,b,,,,,,,
6,a,b,,,,,,,
7,a,b,,,,,,,
8,a,b,,,,,,,


In [71]:
new_df_ltr = np.array(df_ltr.sum(axis=1))
new_ltr = []
# ORDERING OF THE LETTERS!!!
for i in range(len(group_order)):
  ltr = [group_order[i], thsd.groupsunique.tolist().index(group_order[i]),
  new_df_ltr[thsd.groupsunique.tolist().index(group_order[i])]]
  print(ltr)
  new_ltr.append(ltr[2])
print(np.array(df_ltr.sum(axis=1)), new_ltr)

['n1', 0, 'ab']
['n2', 1, 'ab']
['n3', 2, 'ab']
['n4', 3, 'ab']
['n5', 4, 'a']
['n6', 5, 'b']
['n7', 6, 'ab']
['n8', 7, 'ab']
['n9', 8, 'ab']
['ab' 'ab' 'ab' 'ab' 'a' 'b' 'ab' 'ab' 'ab'] ['ab', 'ab', 'ab', 'ab', 'a', 'b', 'ab', 'ab', 'ab']
